<h1><center>D209 Performance Assessment - Data Mining I</center></h1>
<h3><center> by Bader Ale <center><h3>

# ▶ Research Question
Our research questions is as follows: __What patient factors contributed to the highest total charge billed to the patient during their hospital stay.__.

In [1]:
# Importing libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.interactiveshell import InteractiveShell # Importing so we can run multiple lines in one cell
InteractiveShell.ast_node_interactivity = "all" # Code so multiple lines in one cell can be ran simultaenously 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing original dataset
df = pd.read_csv('F:/GitHub Repos/WGU_MSDA/D208_Predictive Modeling/medical_clean.csv')

# ▶ Data Cleaning  

## Detection and Treatment of Nulls

## Detection and Treatment of Duplicated Values

## Detection and Treatment of Outliers

# ▶ Exploratory Data Analysis  